Data Loaders Test etc

In [1]:
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline')
import os
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing
from torch.utils.data import DataLoader

In [25]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Town10HD_Scenario10_route9_11_28_17_59_39 has varying number of data files among input folders. It got discarded from the dataset.


Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os

batch_size = 16
learning_rate = 1e-3

transforms = transforms.Compose(
[
    transforms.ToTensor()
])


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
net = torchvision.models.resnet18(pretrained=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Scaling


In [5]:
def normalize_img(tensor):
    v_min, v_max = tensor.min(), tensor.max()
    new_min, new_max = 0, 1
    v_p = (tensor - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    return v_p

Training

In [6]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

num_ftrs = net.fc.in_features

net.fc = nn.Sequential(
    nn.Linear(num_ftrs, 3), #nbr of outputs
    nn.Tanh()
)
net.fc = net.fc.cuda() if device else net.fc

In [7]:
import time

In [8]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 35 sekunden
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
at = time.time()
for batch_idx, data in enumerate(train_dataloader):
    data_ = normalize_img(data["rgb"].transpose(1,4)[:,:,:,:,0]).float()
    target_ = torch.stack([data["speed"],data["steer"],data["throttle"],data["brake"]])[:,:,0].transpose(0,1).float()
et = time.time()

In [9]:
print(et-at)

44.541213035583496


In [26]:
%%time

n_epochs = 3
#print_every = 10
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader):

        # further preprocessing
        data_ = normalize_img(data["rgb"].transpose(1,4)[:,:,:,:,0]).float() # HOTFIX
        target_ = torch.stack([data["steer"],data["throttle"],data["brake"]])[:,:,0].transpose(0,1).float() # HOTFIX
        
        #print(target_.shape)
        #print(target_.shape[0])
        if target_.shape[0] != 16: # HOTFIX
            run = False
            break

        # move to GPU
        data_, target_ = data_.to(device), target_.to(device)
        
        # compute outputs
        optimizer.zero_grad()
        outputs = net(data_)
        loss = criterion(outputs, target_)
        
        # Backprop
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    if not run: # HOTFIX
        continue
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')

    net.train()
    

Epoch 1

Epoch [1/3], Step [0/126], Loss: 0.4166
Epoch [1/3], Step [16/126], Loss: 0.3593
Epoch [1/3], Step [32/126], Loss: 0.3781
Epoch [1/3], Step [48/126], Loss: 0.5881
Epoch [1/3], Step [64/126], Loss: 0.4094
Epoch [1/3], Step [80/126], Loss: 0.4436
Epoch [1/3], Step [96/126], Loss: 0.3413
Epoch [1/3], Step [112/126], Loss: 0.5100
Epoch 2

Epoch [2/3], Step [0/126], Loss: 0.5273
Epoch [2/3], Step [16/126], Loss: 0.5232
Epoch [2/3], Step [32/126], Loss: 0.3299
Epoch [2/3], Step [48/126], Loss: 0.3520
Epoch [2/3], Step [64/126], Loss: 0.3762
Epoch [2/3], Step [80/126], Loss: 0.5340
Epoch [2/3], Step [96/126], Loss: 0.2448
Epoch [2/3], Step [112/126], Loss: 0.4219
Epoch 3

Epoch [3/3], Step [0/126], Loss: 0.3867
Epoch [3/3], Step [16/126], Loss: 0.3404
Epoch [3/3], Step [32/126], Loss: 0.5966
Epoch [3/3], Step [48/126], Loss: 0.4310
Epoch [3/3], Step [64/126], Loss: 0.4956
Epoch [3/3], Step [80/126], Loss: 0.5454
Epoch [3/3], Step [96/126], Loss: 0.3740
Epoch [3/3], Step [112/126], Lo

Test predictions

In [29]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(next(iter(test_dataloader)).keys())
idx, X = next(enumerate(test_dataloader))

dict_keys(['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])


In [37]:
data_ = normalize_img(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = torch.stack([X["steer"],X["throttle"],X["brake"]])[:,:,0].transpose(0,1).float()

In [38]:
outputs_ = net(data_)

In [39]:
print(outputs_[:,:])

tensor([[ 0.3560,  0.1775, -0.5901],
        [ 0.2650,  0.2957, -0.5650],
        [ 0.3745,  0.2124, -0.3028],
        [ 0.3368,  0.2746, -0.5516],
        [-0.0524,  0.2554, -0.4258],
        [ 0.0546,  0.1692, -0.2963],
        [ 0.1007,  0.1127, -0.7916],
        [ 0.3547,  0.1560, -0.6911],
        [ 0.3118,  0.4575, -0.1918],
        [ 0.3185,  0.4312, -0.7122],
        [ 0.2761,  0.4939, -0.5897],
        [ 0.3806,  0.3001, -0.6844],
        [-0.0239,  0.3490, -0.7448],
        [ 0.1096,  0.1025, -0.7726],
        [ 0.3418,  0.3548, -0.6831],
        [ 0.1886,  0.3523, -0.5829]], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [40]:
print(target_)

tensor([[ 0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-3.0000e-03,  5.3349e-02,  0.0000e+00],
        [-0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-5.0000e-03,  7.5000e-01,  0.0000e+00],
        [-1.0000e-03,  4.8775e-02,  0.0000e+00],
        [-0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-5.0000e-04,  0.0000e+00,  1.0000e+00],
        [ 0.0000e+00,  4.8888e-01,  0.0000e+00],
        [ 2.3600e-01,  7.5000e-01,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-0.0000e+00,  0.0000e+00,  1.0000e+00],
        [-4.0000e-03,  7.5000e-01,  0.0000e+00],
        [-0.0000e+00,  1.0734e-01,  0.0000e+00]])


save & load

In [27]:
torch.save(net, 'rgb_resnet.pth')

In [28]:
net = torch.load('rgb_resnet.pth')